In [1]:
import pandas as pd
from os.path import expanduser, join
from glob import glob
import pandas as pd
from datetime import date
from numpy import where
import numpy as np
today = str(date.today())
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
#Set paths
userhome='/Users/lucindasisk/Documents/qT1_10.2020Analysis'
home = '/Volumes/group/proc/TIGERanalysis/qT1'
dest = home + '/Results/Raw_VoxelData2021'
datadir = userhome +'/data/2_Extracted_Stats'
flist = glob(datadir + '/*/ROI_voxel*GlobalR1*')

In [3]:
## Read in and reorient raw output from FSLMeanTS 

In [ ]:
def fix_files(file1):
    inf = file1.replace(datadir, '').replace('/ROI_voxel-contrasts', '').replace('.txt', '')
    print(inf)
    file1 = pd.read_csv(file1, header=None, sep = '  ', engine='python')
    file = file1.transpose().rename(columns = {0:'L/R', 1:'A/P', 2:'S/I', 3:'Voxel Contrast'})
    outfile = file.to_csv(dest + '/{}.csv'.format(inf), index = False)

for x in range(79,len(flist)):
    file1 = flist[x]
    print('Fixing file {} of {}'.format(x, len(flist)))
    fix_files(file1)


In [4]:
# lamyglist = glob(dest + '/*_LeftAmygdala.csv')
# lcaudlist = glob(dest + '/*_LeftCaudate.csv')
# lhipplist = glob(dest + '/*_LeftHippocampus.csv')
# lNAcclist = glob(dest + '/*_LeftNAcc.csv')
# lputlist = glob(dest + '/*_LeftPutamen.csv')
# ramyglist = glob(dest + '/*_RightAmygdala.csv')
# rcaudlist = glob(dest + '/*_RightCaudate.csv')
# rhipplist = glob(dest + '/*_RightHippocampus.csv')
# rNAcclist = glob(dest + '/*_RightNAcc.csv')
# rputlist = glob(dest + '/*_RightPutamen.csv')

# lcclist = glob(dest + '/*_LeftCC.csv')
# rcclist = glob(dest + '/*_RightCC.csv')
# # lcstlist = glob(dest + '/*_LeftCST.csv')
# # rcstlist = glob(dest + '/*_RightCST.csv')
# lunclist = glob(dest + '/*_LeftUF.csv')
# runclist = glob(dest + '/*_RightUF.csv')
# ccmajlist = glob(dest + '/*_CCMaj.csv')
# ccminlist = glob(dest + '/*_CCMin.csv')
global_r1list = glob(dest + '/*_GlobalR1.csv')
llist= [global_r1list]  #lcclist, rcclist, lunclist, runclist, ccmajlist, ccminlist] #lcstlist, rcstlist

# llist = [lamyglist, lcaudlist, lhipplist, lNAcclist, lputlist,
#         ramyglist, rcaudlist, rhipplist, rNAcclist, rputlist]

In [5]:
def compute_mean(tractlist):
    mean_prop = []
    for file in tractlist:
        v1 = pd.read_csv(file, header=0)
        sub = file.replace(dest+'/','').replace('.csv', '') 
        numzs = len(v1[v1['Voxel Contrast'] == 0.0])
        numvs = len(v1)
        v1['Voxel Contrast'] = v1['Voxel Contrast'].replace(0.0, np.nan) #Replace 0's with NaNs in tract masks
        assert len(v1[v1['Voxel Contrast'] == 0.0]) == 0
        mean = np.nanmean(v1['Voxel Contrast'])
        prop_0vox = numzs/numvs #Get proportion of overhanging voxels
        v2 = [sub, mean, prop_0vox]
        mean_prop.append(v2)
    return mean_prop

In [7]:
# Get Global R1 data
gr1means = compute_mean(global_r1list)
gr1file = pd.DataFrame(gr1means, columns = ['Global_T1', 'Global_T1_VoxelConMean', 'Global_T1_zerovoxels'])
assert len(gr1file) == len(flist)
gr1id = []
for x in gr1file['Global_T1']:
    y = str(x).replace('_GlobalR1','')
    gr1id.append(y)
gr1file['ID'] = gr1id
gr1file['Global_R1'] = 1/gr1file['Global_T1_VoxelConMean']



In [8]:
# gr1file.sort_values(by = 'Global_T1').to_csv('/Users/lucindasisk/Box/LS_Folders/SNAPLab/Projects/TIGER_qT1/Data/Global_R1_n{}_{}.csv'.format(len(gr1file), today))

### Extract Data for DTI Tracts

In [7]:
# Cingulum Cingulate

lccmeans = compute_mean(lcclist)
lccfile = pd.DataFrame(lccmeans, columns = ['Left_CC', 'LCC_VoxelConMean', 'LCC_overhang'])
lccid = []
for x in lccfile['Left_CC']:
    y = str(x).replace('_LeftCC','')
    lccid.append(y)
lccfile['ID'] = lccid
lccfile['LCC_R1'] = 1/lccfile['LCC_VoxelConMean']



rccmeans = compute_mean(rcclist)
rccfile = pd.DataFrame(rccmeans, columns = ['Right_CC', 'RCC_VoxelConMean', 'RCC_overhang'])
rccid = []
for x in rccfile['Right_CC']:
    y = str(x).replace('_RightCC','')
    rccid.append(y)
rccfile['ID'] = rccid
rccfile['RCC_R1'] = 1/rccfile['RCC_VoxelConMean']

# # Corticospinal

# means = []
# for file in lcstlist:
#     compute_mean(file)
    
# lcstfile = pd.DataFrame(means, columns = ['Left_CST', 'LCST_VoxelConMean'])
# lcstid = []
# for x in lcstfile['Left_CST']:
#     y = str(x).replace('_LeftCST','')
#     lcstid.append(y)
# lcstfile['ID'] = lcstid
# lcstfile['LCST_R1'] = 1/lcstfile['LCST_VoxelConMean']


# means = []
# for file in rcstlist:
#     compute_mean(file)
  
# rcstfile = pd.DataFrame(means, columns = ['Right_CST', 'RCST_VoxelConMean'])
# rcstid = []
# for x in rcstfile['Right_CST']:
#     y = str(x).replace('_RightCST','')
#     rcstid.append(y)
# rcstfile['ID'] = rcstid
# rcstfile['RCST_R1'] = 1/rcstfile['RCST_VoxelConMean']
    
# Uncinate Fasciculus
luncmeans = compute_mean(lunclist)
luncfile = pd.DataFrame(luncmeans, columns = ['Left_Unc', 'LUF_VoxelConMean', 'LUF_overhang'])
luncid = []
for x in luncfile['Left_Unc']:
    y = str(x).replace('_LeftUF','')
    luncid.append(y)
luncfile['ID'] = luncid
luncfile['LUF_R1'] = 1/luncfile['LUF_VoxelConMean']


runcmeans = compute_mean(runclist)
runcfile = pd.DataFrame(runcmeans, columns = ['Right_Unc', 'RUF_VoxelConMean', 'RUF_overhang'])
runcid = []
for x in runcfile['Right_Unc']:
    y = str(x).replace('_RightUF','')
    runcid.append(y)
runcfile['ID'] = runcid
runcfile['RUF_R1'] = 1/runcfile['RUF_VoxelConMean']

#Corpus Callosum Major/Minor

ccmajmeans = compute_mean(ccmajlist)
ccmajfile = pd.DataFrame(ccmajmeans, columns = ['CC_Maj', 'CCMaj_VoxelConMean', 'CCMaj_overhang'])
ccmajid = []
for x in ccmajfile['CC_Maj']:
    y = str(x).replace('_CCMaj','')
    ccmajid.append(y)
ccmajfile['ID'] = ccmajid
ccmajfile['CCMaj_R1'] = 1/ccmajfile['CCMaj_VoxelConMean']

ccminmeans = compute_mean(ccminlist)
ccminfile = pd.DataFrame(ccminmeans, columns = ['CC_Min', 'CCMin_VoxelConMean', 'CCMin_overhang'])
ccminid = []
for x in ccminfile['CC_Min']:
    y = str(x).replace('_CCMin','')
    ccminid.append(y)
ccminfile['ID'] = ccminid
ccminfile['CCMin_R1'] = 1/ccminfile['CCMin_VoxelConMean']

In [8]:
m1 = pd.merge(lccfile, rccfile, how='outer', on='ID')
m2 = pd.merge(m1, luncfile, how='outer', on='ID')
m3 = pd.merge(m2, runcfile, how='outer', on='ID')
m4 = pd.merge(m3, ccmajfile, how='outer', on='ID')
m5 = pd.merge(m4, ccminfile, how='outer', on='ID')
# m4 = pd.merge(m3, lcstfile, how='outer', on='ID')
# m5 = pd.merge(m4, rcstfile, how='outer', on='ID')


master1=m5

In [10]:
cols_fixed = ['ID', 'LCC_R1', 'LCC_overhang', 'RCC_R1', 'RCC_overhang', 'LUF_R1', 'LUF_overhang','RUF_R1', 'RUF_overhang',
              'CCMaj_R1', 'CCMaj_overhang', 'CCMin_R1','CCMin_overhang']    
master = master1[cols_fixed].sort_values(by='ID')
master['ID'] = master['ID'].str.replace('159x-T3', '159-T3') #Replace rescan x with original ID for merging
master = master.set_index('ID').drop('42-T1', axis=0).reset_index() #Drop TIGER 42-T1 due to severe autism

today = str(date.today())
master.to_csv(home+'/Results/qT1_output_'+today+'.csv', index=False)

In [32]:
#Read in global R1 values
t1vals = pd.read_csv(home + '/Results/Global_R1.txt', sep = '|', header = None)
t1vals.columns = ['ID', 'T1']
t1vals['Global_R1'] = 1/t1vals['T1']
t1vals.to_csv(home+'/Results/globalT1_output_'+today+'.csv', index=False)

### Extract Data from Subcortical Regions

In [19]:
# #Get data for left amygdala
# means = []
# for file in lamyglist:
#     compute_mean(file)
    
# lamygfile = pd.DataFrame(means, columns = ['Left_Amyg', 'LAmyg_VoxelConMean'])
# lamygid = []
# for x in lamygfile['Left_Amyg']:
#     y = str(x).replace('_LeftAmygdala','')
#     lamygid.append(y)
# lamygfile['ID'] = lamygid
# lamygfile['LAmyg_R1'] = 1/lamygfile['LAmyg_VoxelConMean']

# #Get data for right amygdala
# means = []    
# for file in ramyglist:
#     compute_mean(file)

    
# ramygfile = pd.DataFrame(means, columns = ['Right_Amyg', 'RAmyg_VoxelConMean'])
# ramygid = []
# for x in ramygfile['Right_Amyg']:
#     y = str(x).replace('_RightAmygdala','')
#     ramygid.append(y)
# ramygfile['ID'] = ramygid
# ramygfile['RAmyg_R1'] = 1/ramygfile['RAmyg_VoxelConMean']

# #Get data from left caudate
# means = []
# for file in lcaudlist:
#     compute_mean(file)

# lcaudfile = pd.DataFrame(means, columns = ['Left_Caudate', 'LCaudate_VoxelConMean'])
# lcaudid = []
# for x in lcaudfile['Left_Caudate']:
#     y = str(x).replace('_LeftCaudate','')
#     lcaudid.append(y)
# lcaudfile['ID'] = lcaudid
# lcaudfile['LCaudate_R1'] = 1/lcaudfile['LCaudate_VoxelConMean']

# #Get data from right caudate
# means = []
# for file in rcaudlist:
#     compute_mean(file)

    
# rcaudfile = pd.DataFrame(means, columns = ['Right_Caudate', 'RCaudate_VoxelConMean'])
# rcaudid = []
# for x in rcaudfile['Right_Caudate']:
#     y = str(x).replace('_RightCaudate','')
#     rcaudid.append(y)
# rcaudfile['ID'] = rcaudid
# rcaudfile['RCaudate_R1'] = 1/rcaudfile['RCaudate_VoxelConMean']

# #Get data from left hippocampus
# means = []
# for file in lhipplist:
#     compute_mean(file)
    
# lhippfile = pd.DataFrame(means, columns = ['Left_Hipp', 'LHipp_VoxelConMean'])
# lhippid = []
# for x in lhippfile['Left_Hipp']:
#     y = str(x).replace('_LeftHippocampus','')
#     lhippid.append(y)
# lhippfile['ID'] = lhippid
# lhippfile['LHipp_R1'] = 1/lhippfile['LHipp_VoxelConMean']

# #Get data from right hippocampus
# means = []
# for file in rhipplist:
#     compute_mean(file)
    
# rhippfile = pd.DataFrame(means, columns = ['Right_Hipp', 'RHipp_VoxelConMean'])
# rhippid = []
# for x in rhippfile['Right_Hipp']:
#     y = str(x).replace('_RightHippocampus','')
#     rhippid.append(y)
# rhippfile['ID'] = rhippid
# rhippfile['RHipp_R1'] = 1/rhippfile['RHipp_VoxelConMean']

# #Get data from Left Putamen
# means = []
# for file in lputlist:
#     compute_mean(file)
    
# lputfile = pd.DataFrame(means, columns = ['Left_Putamen', 'LPut_VoxelConMean'])
# lputid = []
# for x in lputfile['Left_Putamen']:
#     y = str(x).replace('_LeftPutamen','')
#     lputid.append(y)
# lputfile['ID'] = lputid
# lputfile['LPut_R1'] = 1/lputfile['LPut_VoxelConMean']

# #Get data from Right Putamen
# means = []
# for file in rputlist:
#     compute_mean(file)

# rputfile = pd.DataFrame(means, columns = ['Right_Putamen', 'RPut_VoxelConMean'])
# rputid = []
# for x in rputfile['Right_Putamen']:
#     y = str(x).replace('_RightPutamen','')
#     rputid.append(y)
# rputfile['ID'] = rputid
# rputfile['RPut_R1'] = 1/rputfile['RPut_VoxelConMean']

# #Get data from Left NAcc
# means = []
# for file in lNAcclist:
#     compute_mean(file)
    
# lNAccfile = pd.DataFrame(means, columns = ['Left_NAcc', 'LNAcc_VoxelConMean'])
# lNAccid = []
# for x in lNAccfile['Left_NAcc']:
#     y = str(x).replace('_LeftNAcc','')
#     lNAccid.append(y)
# lNAccfile['ID'] = lNAccid
# lNAccfile['LNAcc_R1'] = 1/lNAccfile['LNAcc_VoxelConMean']

# #Get data from Right NAcc
# means = []
# for file in rNAcclist:
#     compute_mean(file)
    
# rNAccfile = pd.DataFrame(means, columns = ['Right_NAcc', 'RNAcc_VoxelConMean'])
# rNAccid = []
# for x in rNAccfile['Right_NAcc']:
#     y = str(x).replace('_RightNAcc','')
#     rNAccid.append(y)
# rNAccfile['ID'] = rNAccid
# rNAccfile['RNAcc_R1'] = 1/rNAccfile['RNAcc_VoxelConMean']

In [25]:
# m1 = pd.merge(lamygfile, ramygfile, how='outer', on='ID')
# m2 = pd.merge(m1, lhippfile, how='outer', on='ID')
# m3 = pd.merge(m2, rhippfile, how='outer', on='ID')
# m4 = pd.merge(m3, lcaudfile, how='outer', on='ID')
# m5 = pd.merge(m4, rcaudfile, how='outer', on='ID')
# m6 = pd.merge(m5, lputfile, how='outer', on='ID')
# m7 = pd.merge(m6, rputfile, how='outer', on='ID')
# m8 = pd.merge(m7, lNAccfile, how='outer', on='ID')
# m9 = pd.merge(m8, rNAccfile, how='outer', on='ID')

In [29]:
# #Drop non-R1 columns
# #master1 = m3.drop(['Right_CC', 'Left_CC', 'Right_Unc', 'Left_Unc'], axis=1)
# # master1 = m3.drop(['Left_Amyg', 'Right_Amyg', 'Left_Hipp', 'Right_Hipp',
# #                   'Left_Putamen', 'Right_Putamen', 'Left_Caudate', 'Right_Caudate',
# #                   'Left_NAcc', 'Right_NAcc'], axis=1)
# # cols = master1.columns.tolist()
# #Reorder columns (?)\

# master1=m9
# cols_fixed = ['ID', 'LAmyg_VoxelConMean', 'LAmyg_R1', 'RAmyg_VoxelConMean', 'RAmyg_R1', 
#               'LHipp_VoxelConMean', 'LHipp_R1', 'RHipp_VoxelConMean', 'RHipp_R1',
#               'LCaudate_VoxelConMean', 'LCaudate_R1', 'RCaudate_VoxelConMean', 'RCaudate_R1',
#               'LPut_VoxelConMean', 'LPut_R1', 'RPut_VoxelConMean', 'RPut_R1',
#               'LNAcc_VoxelConMean', 'LNAcc_R1', 'RNAcc_VoxelConMean', 'RNAcc_R1'
#              ]    
# master = master1[cols_fixed].sort_values(by='ID')

In [ ]:
# master.to_csv(home+'/Results/qT1_output_'+today+'.csv', index=False)